In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sqlite3
import pandas as pd
from src.tables import MapTable, df_to_tmp_table, df_to_table, get_machines
from src.preprocess import preprocess
from src.utils import get_tables, get_tbl_info, table_to_df, time_utc_now, np_min_chunk

In [10]:
conn = sqlite3.connect('../data/vast.db')
first = conn.execute('SELECT timestamp FROM reliability_ts LIMIT 1').fetchall()[0][0]
last = conn.execute('SELECT timestamp FROM reliability_ts ORDER BY ROWID DESC LIMIT 1').fetchall()[0][0]
print(pd.to_timedelta((last - first) * 1e9))
conn.close()

1 days 12:30:21


In [11]:
import os
os.getcwd()

'C:\\Users\\ergot\\DataspellProjects\\vast\\notebooks'

In [6]:
df = get_machines()
preprocess(df)

In [7]:
conn = sqlite3.connect('test/test.db')

OperationalError: unable to open database file

In [8]:
conn.execute('DROP TABLE tmp_machines')
df_to_table(df, 'tmp_machines', conn)
conn.commit()

NameError: name 'conn' is not defined

In [55]:
conn.commit()

In [80]:
import os
import psutil

The CPU usage is :  0.16666666666666669


In [88]:
# Getting loadover15 minutes
load1, load5, load15 = psutil.getloadavg()

cpu_usage = (load15/os.cpu_count()) * 100

print("The CPU usage is : ", load1)

The CPU usage is :  0.05


In [89]:
import platform
platform.processor()

'AMD64 Family 23 Model 113 Stepping 0, AuthenticAMD'

In [91]:
psutil.cpu_freq()

scpufreq(current=3593.0, min=0.0, max=3593.0)

In [93]:
psutil.cpu_percent(interval=1)

6.1

In [76]:
os.cpu_count()

24

In [28]:
conn.execute('CREATE TABLE tmp (id, mid)')

OperationalError: table tmp already exists

In [32]:
conn.execute('INSERT INTO tmp VALUES (1, 2)').fetchall()
# conn.close()

[]

In [35]:
conn.execute('DROP TABLE IF EXISTS map_tbl').fetchall()

[]

In [37]:
conn.commit()
get_tables(conn)

[]

In [42]:
conn.close()

In [9]:
conn.execute('DROP TABLE IF EXISTS map_tbl')
map_tbl = MapTable(
    'map_tbl',
    'tmp',
    ['id', 'mid']
)
map_tbl.init_db(conn)
assert 'map_tbl' in get_tables(conn)
info = get_tbl_info('map_tbl', conn)
assert info.loc[info.name == 'id', 'pk'].all()
assert (info.name == 'mid').any()

In [54]:
(info.name == 'mid').any()

True

In [46]:
all(info.loc[info.name == 'id', 'pk'])

True

In [69]:
get_tables(conn)

['host_machine_map',
 'online_ts',
 'online_snp',
 'reliability_ts',
 'reliability_snp',
 'hosting_type_ts',
 'hosting_type_snp',
 'static_ip_ts',
 'static_ip_snp',
 'end_date_ts',
 'end_date_snp',
 'compute_cap_ts',
 'compute_cap_snp',
 'total_flops_ts',
 'total_flops_snp',
 'has_avx_ts',
 'has_avx_snp',
 'cpu_cores_ts',
 'cpu_cores_snp',
 'cpu_ram_ts',
 'cpu_ram_snp',
 'disk_space_ts',
 'disk_space_snp',
 'num_gpus_ts',
 'num_gpus_snp',
 'pci_gen_ts',
 'pci_gen_snp',
 'gpu_lanes_ts',
 'gpu_lanes_snp',
 'gpu_ram_ts',
 'gpu_ram_snp',
 'bw_nvlink_ts',
 'bw_nvlink_snp',
 'verification_ts',
 'verification_snp',
 'storage_cost_ts',
 'storage_cost_snp',
 'inet_up_cost_ts',
 'inet_up_cost_snp',
 'inet_down_cost_ts',
 'inet_down_cost_snp',
 'credit_discount_max_ts',
 'credit_discount_max_snp',
 'cpu_name_ts',
 'cpu_name_snp',
 'disk_name_ts',
 'disk_name_snp',
 'mobo_name_ts',
 'mobo_name_snp',
 'gpu_name_ts',
 'gpu_name_snp',
 'cuda_max_good_ts',
 'cuda_max_good_snp',
 'driver_version_ts',
 

In [70]:
table_to_df('reliability_ts', conn)

name,machine_id,reliability,timestamp
0,11815,6000,1690464352
1,11814,6000,1690464352
2,11813,5727,1690464352
3,11812,9026,1690464352
4,11811,7779,1690464352
...,...,...,...
37259,5456,8515,1690522270
37260,4795,8063,1690522270
37261,4558,9821,1690522270
37262,4557,9848,1690522270


In [71]:
conn.close()

In [57]:
machines = get_machines()
preprocess(machines)
df_to_tmp_table(machines, 'tmp_machines', conn)

In [61]:
machines.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1209 entries, 0 to 1208
Data columns (total 57 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   bw_nvlink             1209 non-null   int32  
 1   compute_cap           1209 non-null   int64  
 2   cpu_cores             1209 non-null   int32  
 3   cpu_name              1209 non-null   object 
 4   cpu_ram               1209 non-null   int32  
 5   credit_balance        1209 non-null   object 
 6   credit_discount       1209 non-null   object 
 7   credit_discount_max   1209 non-null   int32  
 8   cuda_max_good         1209 non-null   object 
 9   direct_port_count     1209 non-null   int64  
 10  discount_rate         1209 non-null   float64
 11  discounted_dph_total  1209 non-null   float64
 12  discounted_hourly     1209 non-null   int64  
 13  disk_bw               1209 non-null   float64
 14  disk_name             1209 non-null   object 
 15  disk_space           

In [47]:
name = 'reliability2'
table = NewTS(
    name,
    'tmp_machines',
    ['machine_id', name]
)

In [48]:
table.init_db(conn)

In [52]:
table.write_db(conn)

[autoreload of _pydevd_bundle.pydevd_frame failed: Traceback (most recent call last):
  File "C:\Users\ergot\mambaforge\envs\ml\lib\site-packages\IPython\extensions\autoreload.py", line 273, in check
    superreload(m, reload, self.old_objects)
  File "C:\Users\ergot\mambaforge\envs\ml\lib\site-packages\IPython\extensions\autoreload.py", line 471, in superreload
    module = reload(module)
  File "C:\Users\ergot\mambaforge\envs\ml\lib\importlib\__init__.py", line 169, in reload
    _bootstrap._exec(spec, module)
  File "<frozen importlib._bootstrap>", line 612, in _exec
  File "<frozen importlib._bootstrap>", line 556, in _init_module_attrs
  File "<frozen importlib._bootstrap>", line 397, in cached
  File "<frozen importlib._bootstrap_external>", line 513, in _get_cached
  File "<frozen importlib._bootstrap_external>", line 406, in cache_from_source
  File "<frozen importlib._bootstrap_external>", line 134, in _path_split
  File "<frozen importlib._bootstrap_external>", line 134, in <